# Upload dataset (contains title/tags/heading/source/text/bias rating for ~20,000 articles) and explore counts across topic, bias rating

In [ ]:
#Upload file

from google.colab import files

uploaded = files.upload()
print(uploaded.keys())

import io
import pandas as pd

df = pd.read_csv(io.BytesIO(uploaded['all_sides_2022.csv']))

Saving all_sides_2022.csv to all_sides_2022.csv
dict_keys(['all_sides_2022.csv'])


In [ ]:
#Peek at df
df

,Unnamed: 0,title,tags,heading,source,text,bias_rating
0,0,Gun Violence Over Fourth of July Weekend,"['Protests', 'Fourth Of July', 'Gun Control An...",Chicago Gun Violence Spikes and Increasingly F...,New York Times (News),As Yasmin Miller drove home from a laundromat ...,left
1,1,Gun Violence Over Fourth of July Weekend,"['Protests', 'Fourth Of July', 'Gun Control An...",‘Bullets just came from nowhere’: Fourth of Ju...,Chicago Tribune,As many Chicagoans were celebrating the Fourth...,center
2,2,Gun Violence Over Fourth of July Weekend,"['Protests', 'Fourth Of July', 'Gun Control An...",Dozens of shootings across US mark bloody July...,New York Post (News),The nation’s 4th of July weekend was marred by...,right
3,3,Yellen Warns Congress of 'Economic Recession' ...,"['Janet Yellen', 'Debt Ceiling', 'Economic Pol...",Federal Government Will Run Out of Cash on Oct...,The Epoch Times,Treasury Secretary Janet Yellen on Tuesday war...,right
4,4,Yellen Warns Congress of 'Economic Recession' ...,"['Janet Yellen', 'Debt Ceiling', 'Economic Pol...",Yellen tells Congress that U.S. will run out o...,Washington Post,Treasury Secretary Janet Yellen on Tuesday tol...,left
...,...,...,...,...,...,...,...
21749,21749,Biden Seeks $30 Billion For Disaster Aid And R...,"['Disaster', 'Joe Biden', 'Disaster Aid', 'Afg...",White House asks Congress to pass emergency fu...,Washington Post,The White House budget office Tuesday sent Con...,left
21750,21750,Biden Seeks $30 Billion For Disaster Aid And R...,"['Disaster', 'Joe Biden', 'Disaster Aid', 'Afg...","Biden wants $30B for disaster aid, Afghan rese...",Washington Examiner,The White House budget office on Tuesday urged...,right
21751,21751,US House Condemns QAnon Group,"['US House', 'QAnon', 'Free Speech', 'Politics']",Seventeen Republican congressmen and Justin Am...,Washington Examiner,The House passed a resolution condemning the c...,right
21752,21752,US House Condemns QAnon Group,"['US House', 'QAnon', 'Free Speech', 'Politics']",Why did some Republicans balk at a resolution ...,NBC News (Online),As many Americans came to realize in recent mo...,left


In [ ]:
#Gauge topic breakdown, most popular topics

from collections import Counter
import ast
import pandas as pd

tag_counts = Counter()

for tags in df['tags']:
    tag_list = ast.literal_eval(tags)
    tag_counts.update(tag_list)

tag_counts_df = pd.DataFrame(tag_counts.items(), columns=['Tag', 'Count']).sort_values(by='Count', ascending=False)

tag_counts_df.head(30)

,Tag,Count
13,Politics,4089
51,Elections,3187
47,Donald Trump,2240
98,Coronavirus,2139
60,World,1936
9,Economy And Jobs,1880
50,Presidential Elections,1698
27,Joe Biden,1597
43,White House,1580
45,Public Health,1047


# Preprocessing

In [ ]:
#Filtered df, by tags. Ultimately chose to use the full set for the sake of a high sample size

specified_tags = {'Coronavirus', 'Public Health', 'Healthcare'}  #adjust as needed

def has_specified_tags(tags):
    tag_list = ast.literal_eval(tags)
    return any(tag in specified_tags for tag in tag_list)

df_filtered = df[df['tags'].apply(has_specified_tags)]

df_filtered.head()

,Unnamed: 0,title,tags,heading,source,text,bias_rating
9,9,Denying Abortion Medication Could Violate Civi...,"['Abortion', 'Pharmacies', 'Abortion Pills', '...",Biden admin to pharmacies: Refusing to fill co...,Politico,Senior Biden administration officials announce...,left
10,10,Denying Abortion Medication Could Violate Civi...,"['Abortion', 'Pharmacies', 'Abortion Pills', '...",Pharmacies risk violating civil rights laws if...,CNBC,The Health and Human Services Department on We...,center
11,11,Denying Abortion Medication Could Violate Civi...,"['Abortion', 'Pharmacies', 'Abortion Pills', '...",HHS warns pharmacies denying abortion pills vi...,Washington Examiner,The Biden administration is warning retail pha...,right
33,33,Trump's Small Financial Stake in Hydroxychloro...,"['Healthcare', 'White House', 'Hydroxychloroqu...",‘Follow the money’: MSNBC contributor blows th...,Raw Story,President Donald Trump has been raising questi...,left
34,34,Trump's Small Financial Stake in Hydroxychloro...,"['Healthcare', 'White House', 'Hydroxychloroqu...","Trump Has ‘Small,’ ‘Distant Link’ To Sanofi, F...",Forbes,Topline: President Trump—increasingly a booste...,center


In [ ]:
#Add general is_biased column for C vs L/R => binary classification

df['is_biased'] = df['bias_rating'].apply(lambda x: 1 if x in ['left', 'right'] else 0)

df

,Unnamed: 0,title,tags,heading,source,text,bias_rating,is_biased
0,0,Gun Violence Over Fourth of July Weekend,"['Protests', 'Fourth Of July', 'Gun Control An...",Chicago Gun Violence Spikes and Increasingly F...,New York Times (News),As Yasmin Miller drove home from a laundromat ...,left,1
1,1,Gun Violence Over Fourth of July Weekend,"['Protests', 'Fourth Of July', 'Gun Control An...",‘Bullets just came from nowhere’: Fourth of Ju...,Chicago Tribune,As many Chicagoans were celebrating the Fourth...,center,0
2,2,Gun Violence Over Fourth of July Weekend,"['Protests', 'Fourth Of July', 'Gun Control An...",Dozens of shootings across US mark bloody July...,New York Post (News),The nation’s 4th of July weekend was marred by...,right,1
3,3,Yellen Warns Congress of 'Economic Recession' ...,"['Janet Yellen', 'Debt Ceiling', 'Economic Pol...",Federal Government Will Run Out of Cash on Oct...,The Epoch Times,Treasury Secretary Janet Yellen on Tuesday war...,right,1
4,4,Yellen Warns Congress of 'Economic Recession' ...,"['Janet Yellen', 'Debt Ceiling', 'Economic Pol...",Yellen tells Congress that U.S. will run out o...,Washington Post,Treasury Secretary Janet Yellen on Tuesday tol...,left,1
...,...,...,...,...,...,...,...,...
21749,21749,Biden Seeks $30 Billion For Disaster Aid And R...,"['Disaster', 'Joe Biden', 'Disaster Aid', 'Afg...",White House asks Congress to pass emergency fu...,Washington Post,The White House budget office Tuesday sent Con...,left,1
21750,21750,Biden Seeks $30 Billion For Disaster Aid And R...,"['Disaster', 'Joe Biden', 'Disaster Aid', 'Afg...","Biden wants $30B for disaster aid, Afghan rese...",Washington Examiner,The White House budget office on Tuesday urged...,right,1
21751,21751,US House Condemns QAnon Group,"['US House', 'QAnon', 'Free Speech', 'Politics']",Seventeen Republican congressmen and Justin Am...,Washington Examiner,The House passed a resolution condemning the c...,right,1
21752,21752,US House Condemns QAnon Group,"['US House', 'QAnon', 'Free Speech', 'Politics']",Why did some Republicans balk at a resolution ...,NBC News (Online),As many Americans came to realize in recent mo...,left,1


In [ ]:
#!pip install spacy
#!python -m spacy download en_core_web_sm

In [ ]:
#Balance the dataset (center, left/right)

min_count = df["is_biased"].value_counts().min()

df_sample = (
    df.groupby("is_biased")
    .apply(lambda x: x.sample(n=min_count, random_state=42))
    .reset_index(drop=True)
)

#Check class distribution
print(df_sample["is_biased"].value_counts())

is_biased
0    4253
1    4253
Name: count, dtype: int64


<ipython-input-7-30c18231e616>:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=min_count, random_state=42))


In [ ]:
import spacy

#Load spaCy English model
nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    doc = nlp(text)  #Process text with spaCy
    tokens = [
        token.lemma_.lower()  #Lemmatize and lowercase
        for token in doc
        if not token.is_stop and not token.is_punct  #Remove stopwords & punctuation
    ]
    return " ".join(tokens)  #Join processed tokens back into a string

#Apply preprocessing to the "heading" column
df_sample["processed_heading"] = df_sample["heading"].astype(str).apply(preprocess_text)
#Apply preprocessing to the "heading" column
df_sample["processed_text"] = df_sample["text"].astype(str).apply(preprocess_text)

#Display the first few rows
df_sample[["heading", "processed_heading", "processed_text"]].head()

,heading,processed_heading,processed_text
0,Trump moves to relax Obama-era water protections,trump move relax obama era water protection,trump administration tuesday propose reduce fe...
1,‘Red Wave’ Optimism Quickly Fizzles Among Repu...,red wave optimism quickly fizzles republicans,donald trump jr early evening prediction blood...
2,DOJ claims Yale discriminates against Asian an...,doj claim yale discriminate asian white applicant,justice department thursday accuse yale univer...
3,Abrams Apologizes For Maskless Photo-Op With S...,abrams apologizes maskless photo op schoolkids...,stacey abrams apologize tuesday appear maskles...
4,U.S. Confirms Death of American Hostage Kayla ...,u.s. confirm death american hostage kayla mueller,kayla jean mueller american hostage know hold ...


# TF-IDF (Bag of Words) + Logistic Regression (Basline) - Text

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df_sample["processed_text"], df_sample["is_biased"], test_size=0.2, random_state=42
)

vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=5000)  # Unigrams & bigrams
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

classifier = LogisticRegression()
classifier.fit(X_train_tfidf, y_train)

y_pred = classifier.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.5987074030552292
              precision    recall  f1-score   support

           0       0.60      0.63      0.61       866
           1       0.60      0.57      0.58       836

    accuracy                           0.60      1702
   macro avg       0.60      0.60      0.60      1702
weighted avg       0.60      0.60      0.60      1702



# TF-IDF (Bag of Words) + Logistic Regression (Baseline) - Headlines

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df_sample["processed_heading"], df_sample["is_biased"], test_size=0.2, random_state=42
)

vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=5000)  #Unigrams & bigrams
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

classifier = LogisticRegression()
classifier.fit(X_train_tfidf, y_train)

y_pred = classifier.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.5517038777908343
              precision    recall  f1-score   support

           0       0.56      0.56      0.56       866
           1       0.54      0.54      0.54       836

    accuracy                           0.55      1702
   macro avg       0.55      0.55      0.55      1702
weighted avg       0.55      0.55      0.55      1702



# Random Forrest Classifier - Headlines

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=5000)  #Unigrams & bigrams, no trigrams
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_tfidf, y_train)
y_pred_rf = rf_classifier.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

Accuracy: 0.5411280846063454
              precision    recall  f1-score   support

           0       0.54      0.63      0.58       866
           1       0.54      0.45      0.49       836

    accuracy                           0.54      1702
   macro avg       0.54      0.54      0.54      1702
weighted avg       0.54      0.54      0.54      1702



# SVM - Headlines

In [ ]:
svm_classifier = SVC(kernel="linear", probability=True, random_state=42)  #Linear SVM
svm_classifier.fit(X_train_tfidf, y_train)
y_pred_svm = svm_classifier.predict(X_test_tfidf)

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))

Accuracy: 0.5511163337250293
              precision    recall  f1-score   support

           0       0.56      0.56      0.56       866
           1       0.54      0.54      0.54       836

    accuracy                           0.55      1702
   macro avg       0.55      0.55      0.55      1702
weighted avg       0.55      0.55      0.55      1702



In [ ]:
#Find optimal SVC parameters systematically

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': [0.1, 1, 10, 100],  #Regularization strength
    'kernel': ['linear', 'rbf'],  #Linear vs. non-linear kernel
    'gamma': ['scale', 'auto']  #Only used for 'rbf' kernel
}

svm_model = SVC()

#Grid search with 5-fold cross-validation
grid_search = GridSearchCV(svm_model, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
grid_search.fit(X_train_tfidf, y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)

best_svm = SVC(**grid_search.best_params_)
best_svm.fit(X_train_tfidf, y_train)

y_pred_best_svm = best_svm.predict(X_test_tfidf)

print("\nOptimized SVM Performance:")
print("Accuracy:", accuracy_score(y_test, y_pred_best_svm))
print(classification_report(y_test, y_pred_best_svm))


Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best Parameters: {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}
Best Accuracy: 0.5558484246012879

Optimized SVM Performance:
Accuracy: 0.5470035252643948
              precision    recall  f1-score   support

           0       0.55      0.57      0.56       866
           1       0.54      0.52      0.53       836

    accuracy                           0.55      1702
   macro avg       0.55      0.55      0.55      1702
weighted avg       0.55      0.55      0.55      1702



# Fine-Tuned DistilBert Model

In [ ]:
#!pip install transformers torch shap

In [ ]:
#!pip install datasets

In [ ]:
import torch
from transformers import DistilBertTokenizerFast

In [ ]:
from datasets import Dataset, ClassLabel

In [ ]:
import torch
import pandas as pd
import numpy as np
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
#Convert DataFrame to Hugging Face Dataset format
df_sample["label"] = df_sample["is_biased"].astype(int)  # Rename target column

train_texts, test_texts, train_labels, test_labels = train_test_split(
    df_sample["processed_heading"].tolist(), df_sample["label"].tolist(), test_size=0.2, random_state=42
)

#Load tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

#Convert to Dataset format
train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels})
test_dataset = Dataset.from_dict({"text": test_texts, "label": test_labels})

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/6804 [00:00<?, ? examples/s]

Map:   0%|          | 0/1702 [00:00<?, ? examples/s]

In [ ]:
#Load model with number of output classes = 2 (biased or not)
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",  #DISABLE WANDB
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [ ]:
#!pip install wandb

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.693200,0.694166,0.531140,0.446194,0.601193,0.531140
2,0.604500,0.719626,0.555229,0.546263,0.563169,0.555229
3,0.434900,0.864446,0.564630,0.564373,0.565475,0.564630


TrainOutput(global_step=1278, training_loss=0.5873023342451952, metrics={'train_runtime': 1023.7774, 'train_samples_per_second': 19.938, 'train_steps_per_second': 1.248, 'total_flos': 2703924541366272.0, 'train_loss': 0.5873023342451952, 'epoch': 3.0})

In [ ]:
#Save model, this takes quite a while to train!
trainer.save_model("bert_model_bi")

In [ ]:
results = trainer.evaluate()
results

{'eval_loss': 0.8644458651542664,
 'eval_accuracy': 0.5646298472385429,
 'eval_f1': 0.5643726207714316,
 'eval_precision': 0.5654747989992281,
 'eval_recall': 0.5646298472385428,
 'eval_runtime': 25.8234,
 'eval_samples_per_second': 65.909,
 'eval_steps_per_second': 4.144,
 'epoch': 3.0}

In [ ]:
predictions = trainer.predict(test_dataset)

#Convert logits to class predictions
y_pred = np.argmax(predictions.predictions, axis=1)
y_true = test_dataset["label"]

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=["Not Biased (0)", "Biased (1)"]))

Classification Report:
                precision    recall  f1-score   support

Not Biased (0)       0.58      0.54      0.56       866
    Biased (1)       0.55      0.59      0.57       836

      accuracy                           0.56      1702
     macro avg       0.57      0.57      0.56      1702
  weighted avg       0.57      0.56      0.56      1702

